In [26]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

import tensorflow as tf
from tensorflow import keras as k

import pandas as pd
from utility import *

In [27]:
data = load_data()
X, y = create_XY(data)

enc = OneHotEncoder(sparse=False)
y = enc.fit_transform(y)
X_imputed = SimpleImputer().fit_transform(X)

In [28]:
def time_step(data, label, step=10):
    x,y = [],[]
    for i in range(data.shape[0]-(step+1)):
        x.append(data[i:i+step])
        y.append(label[i+step+1])
    return np.array(x),np.array(y)

In [29]:
def create_dataset(data, team_name):
    ht = data.loc[data['HomeTeam']==team_name]
    at = data.loc[data['AwayTeam']==team_name]
    team_data = pd.concat([ht, at])
    team_data_label = team_data['FTR']
    team_data_featured = pd.get_dummies(team_data[features])
    return team_data_featured, team_data_label

In [30]:
team_name = 'Chelsea'
data_f, data_l = create_dataset(data, team_name)
data_f = SimpleImputer().fit_transform(data_f)
x_ars, y_ars = time_step(data_f, data_l.to_numpy())

print(x_ars.shape)
print(y_ars.shape)

y_ars = enc.fit_transform(y_ars.reshape(-1,1))



(407, 10, 94)
(407,)


In [31]:
x_train, x_test, y_train, y_test = train_test_split(x_ars, y_ars, shuffle=False, test_size=test_size)

Tx = x_train.shape[1] #Time steps
Ty = y_train.shape[1] #Time Steps

# LSTM

In [32]:
import os

model = None
model_path = './models/lstm/'

# Check se il modello è già salvato
if not os.path.exists(model_path) or os.listdir(model_path):
    os.mkdir("./" + model_path.split("/")[1])
    os.mkdir(model_path)


else:
    model = k.models.load_model(model_path)


if model == None:
    model = k.models.Sequential(
        [
            k.layers.LSTM(64, return_sequences=True),
            k.layers.Dropout(0.4),
            k.layers.Dense(1000, activation="relu"),
            k.layers.Dropout(0.3),
            k.layers.Dense(250, activation="relu"),
            k.layers.Dropout(0.2),
            k.layers.Dense(3, activation="softmax")
        ]
    )

    model.compile(
        loss='categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.0001),
        metrics=["accuracy"]
    )

    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

    model.fit(x_train, y_train, epochs=10, callbacks=[callback])
    model.save('./models/lstm')

OSError: SavedModel file does not exist at: ./models/lstm//{saved_model.pbtxt|saved_model.pb}

In [ ]:
model.evaluate(x_test, y_test)
report(model, x_train, y_train, x_test, y_test, enc)

## 2 LSTM concatenate + dense e dropout in sequenza

In [ ]:
inputs = k.layers.Input(shape=(10,94))
lstm1 = k.layers.LSTM(100, return_sequences=True, activation="relu")(inputs)
lstm2 = k.layers.LSTM(50, return_sequences=True, activation="relu")(inputs)
concateneted = k.layers.Concatenate()([
    lstm1,
    lstm2
])

out = k.layers.Dropout(0.5)(concateneted)
out = k.layers.Dense(1000, activation="relu")(out)
out = k.layers.Dropout(0.5)(out)
out = k.layers.Dense(250, activation="relu")(out)
out = k.layers.Dropout(0.5)(out)
out = k.layers.Dense(3, activation="softmax")(out)

model = k.models.Model(inputs=inputs, outputs=out)

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

model.fit(x_train, y_train, epochs=500, callbacks=[callback])

In [ ]:
model.evaluate(x_test, y_test)
report(model, x_train, y_train, x_test, y_test, enc)

### Fit
loss: 0.9373 - accuracy: 0.5609
### Evaluate
loss: 0.9192 - accuracy: 0.5930
### Classification report

| | precision | recall | f1-score | support |
| - | ----------- | ------ | -------- | ------- |
| A | 0.609 | 0.614 | 0.612 | 345 |
| D | 0.321 | 0.201 | 0.247 | 254 |
| H | 0.652 | 0.763 | 0.704 | 541 |
| accuracy | |  | 0.593 | 1140 |
| macro avg | 0.527 | 0.526 | 0.521 | 1140 |
| weighted avg | 0.565 | 0.593 | 0.574 | 1140 |

## LSTM in parallelo

In [ ]:
inputs = k.layers.Input(shape=(10,94))
x = []
for t in range(Tx):
    module = k.layers.LSTM((t+1)*10, return_sequences=True)(inputs)
    module = k.layers.Dropout(0.7)(module)
    x.append(module)
x = k.layers.Concatenate()(x)

out = k.layers.Dense(1000, activation="relu")(x)
out = k.layers.Dropout(0.7)(out)
out = k.layers.Dense(250, activation="relu")(out)
out = k.layers.Dense(3, activation="softmax")(out)

txLstm = k.models.Model(inputs=inputs, outputs=out)

In [ ]:
txLstm.summary()

In [ ]:
txLstm.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

txLstm.fit(x_train_lstm, y_train_lstm, epochs=500, callbacks=[callback], batch_size=16)

In [ ]:
txLstm.evaluate(y_test_lstm, y_test_lstm)
report(txLstm, x_train_lstm, y_train_lstm, x_test_lstm, y_test_lstm, enc)

# GRU

In [ ]:
gru = k.models.Sequential([
    k.layers.GRU(100, return_sequences=True, activation="relu"),
    k.layers.Dropout(0.4),
    k.layers.Dense(1000, activation="relu"),
    k.layers.Dropout(0.3),
    k.layers.Dense(250, activation="relu"),
    k.layers.Dropout(0.2),
    k.layers.Dense(3, activation="softmax")
])

In [ ]:
gru(x_train_lstm)
gru.summary()

In [ ]:
gru.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

gru.fit(x_train_lstm, y_train_lstm, epochs=500, callbacks=[callback])

In [ ]:
gru.evaluate(x_test_lstm, y_test_lstm)
report(gru, x_train_lstm, y_train_lstm, x_test_lstm, y_test_lstm, enc)

# Time distributed

In [ ]:
model = k.models.Sequential(
    [
        k.layers.GRU(128, dropout=0.5, return_sequences=True),
        k.layers.Flatten(),
        k.layers.Dense(40, activation="relu"),
        k.layers.Dropout(0.4),
        k.layers.Dense(20, activation="relu"),
        k.layers.Dropout(0.2),
        k.layers.Dense(10, activation="relu"),
        k.layers.Dense(3, activation="softmax")
    ]
)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-12-15 17:13:25.150328: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-15 17:13:25.150473: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=["accuracy"]
)
model(x_train)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (296, 10, 128)            86016     
                                                                 
 flatten (Flatten)           (296, 1280)               0         
                                                                 
 dense (Dense)               (296, 40)                 51240     
                                                                 
 dropout (Dropout)           (296, 40)                 0         
                                                                 
 dense_1 (Dense)             (296, 20)                 820       
                                                                 
 dropout_1 (Dropout)         (296, 20)                 0         
                                                                 
 dense_2 (Dense)             (296, 10)                 2

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

model.fit(x_train, y_train, epochs=1000, callbacks=[callback])

Epoch 1/1000


2022-12-15 17:13:58.979764: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-15 17:14:00.554430: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 17:14:00.800466: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 1/10 [==>...........................] - ETA: 20s - loss: 1.4210 - accuracy: 0.3438

2022-12-15 17:14:01.096027: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 3s 51ms/step - loss: 1.4587 - accuracy: 0.2466
Epoch 2/1000
10/10 [==============================] - 0s 25ms/step - loss: 1.2842 - accuracy: 0.2534
Epoch 3/1000
10/10 [==============================] - 0s 26ms/step - loss: 1.1451 - accuracy: 0.3480
Epoch 4/1000
10/10 [==============================] - 0s 26ms/step - loss: 1.1114 - accuracy: 0.3784
Epoch 5/1000
10/10 [==============================] - 0s 25ms/step - loss: 1.1144 - accuracy: 0.3750
Epoch 6/1000
10/10 [==============================] - 0s 24ms/step - loss: 1.0633 - accuracy: 0.4324
Epoch 7/1000
10/10 [==============================] - 0s 24ms/step - loss: 1.0677 - accuracy: 0.4662
Epoch 8/1000
10/10 [==============================] - 0s 25ms/step - loss: 1.0598 - accuracy: 0.4696
Epoch 9/1000
10/10 [==============================] - 0s 25ms/step - loss: 1.0094 - accuracy: 0.4865
Epoch 10/1000
10/10 [==============================] - 0s 25ms/step - loss: 1.0654 - accuracy: 0.4865
Ep

In [ ]:
model.evaluate(x_test, y_test)

report(model, x_train, y_train, "train", enc)
report(model, x_test, y_test, "test", enc)

1/4 [======>.......................] - ETA: 1s - loss: 1.0213 - accuracy: 0.5938

2022-12-15 17:14:11.126724: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 17:14:11.207441: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 9/10 [==========================>...] - ETA: 0s

2022-12-15 17:14:11.674269: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-15 17:14:11.736102: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 0s 12ms/step
TRAIN REPORT
              precision    recall  f1-score   support

           A      0.487     0.565     0.523        69
           D      1.000     0.047     0.090        64
           H      0.657     0.859     0.745       163

    accuracy                          0.615       296
   macro avg      0.715     0.490     0.453       296
weighted avg      0.692     0.615     0.551       296

--------------------------------------------------
4/4 [==============================] - 0s 21ms/step
TEST REPORT
              precision    recall  f1-score   support

           A      0.537     0.850     0.658        60
           D      0.000     0.000     0.000        21
           H      0.273     0.100     0.146        30

    accuracy                          0.486       111
   macro avg      0.270     0.317     0.268       111
weighted avg      0.364     0.486     0.395       111

--------------------------------------------------
